In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
raw_df_match_details = pd.read_csv('../data/raw/match_details.csv')
raw_df_player_attr = pd.read_csv('../data/raw/player_attributes.csv')

In [3]:
raw_df_player_attr

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [4]:
# raw_df_match_details.info()

In [5]:
# raw_df_player_attr.info()

In [6]:
raw_df_match_details['date'] = pd.to_datetime(raw_df_match_details['date'])
raw_df_player_attr['date'] = pd.to_datetime(raw_df_player_attr['date'])
raw_df_match_details.sort_values(by=['season', 'date'], inplace=True)

In [7]:
def get_points(row, team):
    points_mapping = {
        'H': {'home': 3, 'away': 0},
        'D': {'home': 1, 'away': 1},
        'A': {'home': 0, 'away': 3}
    }
    team_type = 'home' if row['home_team'] == team else 'away'
    return int(points_mapping[row['result_match']][team_type])

def process_points(team, df, match_date, match_season):
    team_matches = df.query('(home_team == @team | away_team == @team) & (season == @match_season & date < @match_date)')
    if len(team_matches) == 0:
        return 0

    return team_matches.apply(lambda row: get_points(row, team), axis=1).sum()

def count_points(match_row, df):
    match_date = match_row['date']
    match_season = match_row['season']
    home_team = match_row['home_team']
    away_team = match_row['away_team']

    home_team_points = process_points(home_team, df, match_date, match_season)
    away_team_points = process_points(away_team, df, match_date, match_season)

    return home_team_points, away_team_points

raw_df_match_details[['points_home', 'points_away']] = raw_df_match_details.apply(lambda row: count_points(row, raw_df_match_details), axis=1, result_type='expand')

In [15]:
raw_df_match_details = (raw_df_match_details
                        .assign(date=pd.to_datetime(raw_df_match_details['date']),
                                result_match=lambda df: (df['result_match'] == 'H').astype(int))
                        )

In [10]:
from playerstats import player_stats

players_cols = ['{}_player_{}'.format(team, i) for team in ['home', 'away'] for i in range(1, 12)]

player_stats_dict_series = raw_df_match_details.apply(
    lambda row: player_stats.get_player_stat(
        match_row=row,
        df_matches=raw_df_match_details,
        df_player_attr=raw_df_player_attr,
        players=players_cols
    ),
    axis=1
)

new_player_stats_df = pd.json_normalize(player_stats_dict_series)

df = pd.merge(raw_df_match_details, new_player_stats_df, how='left', on='match_api_id')
df.drop(players_cols, axis=1, inplace=True)
matching_columns = [col for col in df.columns if 'player_rating' in col] + [col for col in df.columns if
                                                                            'aggression_rating' in col] + [col for col
                                                                                                           in df.columns
                                                                                                           if
                                                                                                           'strength_rating' in col] + [
                       col for col in df.columns if 'acceleration_rating' in col]
df.dropna(subset=matching_columns, inplace=True)
matching_columns_filtered = [c for c in df.columns if c != "date" and c != "season"]

In [11]:
from src.shiftdata.shift_data import ShiftDataPreprocessor

preprocessor = ShiftDataPreprocessor(df)

home_df = preprocessor.select_and_rename_columns('home_')
away_df = preprocessor.select_and_rename_columns('away_')

team_df = preprocessor.concatenate_teams(home_df, away_df)

features_to_shift = ['team_goal', 'team_shoton', 'team_possession']
team_df_shifted = preprocessor.shift_features(features_to_shift)

df_final = preprocessor.merge_shifted_features(team_df_shifted)

INFO: Merging shifted features back into the original DataFrame.


In [12]:
df_ = df_final.copy()

In [13]:
df_[:5].to_csv('../data/preprocessed/match_details_5.csv', index=False)

In [14]:
df_

,match_api_id,season,stage,date,away_team,home_team,result_match,points_home,points_away,overall_rating_home_player_1,...,overall_rating_away_player_11,acceleration_rating_away_player_11,strength_rating_away_player_11,aggression_rating_away_player_11,home_last_team_goal,home_last_team_shoton,home_last_team_possession,away_last_team_goal,away_last_team_shoton,away_last_team_possession
18,489063,2008/2009,3,2008-08-30,10261,9825,1,3,4,78.500000,...,76.500000,75.000000,81.000000,65.000000,1.0,12.0,34.0,1.0,1.0,55.0
21,489068,2008/2009,3,2008-08-30,10194,8549,1,3,3,73.500000,...,62.000000,72.500000,78.000000,61.000000,1.0,7.0,47.0,3.0,1.0,47.0
22,489069,2008/2009,3,2008-08-30,8659,8559,0,3,0,68.000000,...,68.000000,81.000000,80.000000,66.000000,3.0,5.0,53.0,1.0,2.0,66.0
23,489070,2008/2009,3,2008-08-30,8528,8667,0,4,0,65.500000,...,71.000000,77.000000,77.000000,59.000000,1.0,5.0,47.0,1.0,7.0,52.0
25,489066,2008/2009,3,2008-08-31,8650,10252,0,3,6,78.666667,...,87.333333,91.666667,75.666667,68.333333,2.0,5.0,48.0,2.0,11.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033,1987603,2015/2016,38,2016-05-15,8654,10194,1,48,62,74.600000,...,77.400000,69.000000,92.000000,83.200000,1.0,4.0,47.0,3.0,12.0,52.0
3034,1987604,2015/2016,38,2016-05-15,8456,10003,0,46,65,75.400000,...,87.300000,92.000000,70.900000,57.000000,4.0,9.0,53.0,2.0,4.0,43.0
3035,1987605,2015/2016,38,2016-05-15,8472,9817,0,44,38,72.400000,...,77.700000,75.300000,84.400000,76.100000,2.0,3.0,34.0,3.0,7.0,31.0
3036,1987606,2015/2016,38,2016-05-15,8650,8659,0,42,59,77.800000,...,68.700000,83.600000,61.200000,34.800000,1.0,7.0,54.0,1.0,9.0,66.0


In [15]:
home_strength_rating_cols = df_.filter(like='strength_rating_home').columns.to_list()
away_strength_rating_cols = df_.filter(like='strength_rating_away').columns.to_list()

home_aggression_rating_cols = df_.filter(like='aggression_rating_home').columns.to_list()
away_aggression_rating_cols = df_.filter(like='aggression_rating_away').columns.to_list()

home_overall_rating_cols = df_.filter(like='overall_rating_home').columns.to_list()
away_overall_rating_cols = df_.filter(like='overall_rating_away').columns.to_list()

home_acceleration_rating_cols = df_.filter(like='acceleration_rating_home').columns.to_list()
away_acceleration_rating_cols = df_.filter(like='acceleration_rating_away').columns.to_list()

home_possession_rating_cols = df_.filter(like='home_last_team_possession').columns.to_list()
away_possession_rating_cols = df_.filter(like='away_last_team_possession').columns.to_list()

home_shoton_rating_cols = df_.filter(like='home_last_team_shoton').columns.to_list()
away_shoton_rating_cols = df_.filter(like='away_last_team_shoton').columns.to_list()

In [16]:
df_['team_strength_home'] = df_[home_strength_rating_cols].mean(axis=1)
df_['team_strength_away'] = df_[away_strength_rating_cols].mean(axis=1)
df_['strength_difference'] = df_['team_strength_home'] - df_['team_strength_away']

df_['team_aggression_home'] = df_[home_aggression_rating_cols].mean(axis=1)
df_['team_aggression_away'] = df_[away_aggression_rating_cols].mean(axis=1)
df_['aggression_difference'] = df_['team_aggression_home'] - df_['team_aggression_away']

df_['team_acceleration_home'] = df_[home_acceleration_rating_cols].mean(axis=1)
df_['team_acceleration_away'] = df_[away_acceleration_rating_cols].mean(axis=1)
df_['acceleration_difference'] = df_['team_acceleration_home'] - df_['team_acceleration_away']

df_['points_difference'] = df_['points_home'] - df_['points_away']

df_['rolling_avg_goals_home'] = df_['home_last_team_goal'].rolling(window=5).mean()
df_['rolling_avg_goals_away'] = df_['away_last_team_goal'].rolling(window=5).mean()

df_['goal_conversion_rate_home'] = df_['home_last_team_goal'] / (df_['home_last_team_shoton'] + 1)
df_['goal_conversion_rate_away'] = df_['away_last_team_goal'] / (df_['away_last_team_shoton'] + 1)

columns_to_drop = (
    home_strength_rating_cols
    + away_strength_rating_cols
    + home_aggression_rating_cols
    + away_aggression_rating_cols
    + home_overall_rating_cols
    + away_overall_rating_cols
    + home_acceleration_rating_cols
    + away_acceleration_rating_cols
)

# Drop the valid columns
df_ = df_.drop(columns_to_drop, axis=1)

In [17]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2974 entries, 18 to 3037
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   match_api_id               2974 non-null   int64         
 1   season                     2974 non-null   object        
 2   stage                      2974 non-null   int64         
 3   date                       2974 non-null   datetime64[ns]
 4   away_team                  2974 non-null   int64         
 5   home_team                  2974 non-null   int64         
 6   result_match               2974 non-null   int64         
 7   points_home                2974 non-null   int64         
 8   points_away                2974 non-null   int64         
 9   home_last_team_goal        2974 non-null   float64       
 10  home_last_team_shoton      2974 non-null   float64       
 11  home_last_team_possession  2974 non-null   float64       
 12  away_last_

In [18]:
df_.drop(columns=["match_api_id", "result_match", "season", "stage", "date", "home_team", "away_team"]).describe().round(2)

,points_home,points_away,home_last_team_goal,home_last_team_shoton,home_last_team_possession,away_last_team_goal,away_last_team_shoton,away_last_team_possession,team_strength_home,team_strength_away,...,team_aggression_away,aggression_difference,team_acceleration_home,team_acceleration_away,acceleration_difference,points_difference,rolling_avg_goals_home,rolling_avg_goals_away,goal_conversion_rate_home,goal_conversion_rate_away
count,2974.00,2974.00,2974.00,2974.00,2974.00,2974.00,2974.00,2974.00,2974.00,2974.00,...,2974.00,2974.00,2974.00,2974.00,2974.00,2974.00,2970.00,2970.00,2974.00,2974.00
mean,25.66,25.89,1.81,6.16,49.10,1.93,5.94,50.79,68.50,68.78,...,66.11,-0.50,74.74,74.65,0.09,-0.23,1.81,1.93,0.31,0.34
std,17.78,17.70,0.93,3.32,8.75,1.01,3.18,8.77,6.64,6.52,...,8.53,11.63,5.60,5.37,7.57,13.87,0.41,0.45,0.23,0.26
min,0.00,0.00,1.00,1.00,20.00,1.00,1.00,4.00,45.02,46.90,...,36.23,-41.35,52.31,52.31,-30.68,-54.00,1.00,1.00,0.05,0.05
25%,12.00,12.00,1.00,4.00,43.00,1.00,4.00,45.00,64.45,64.66,...,60.75,-8.08,71.14,71.03,-4.91,-7.00,1.53,1.60,0.17,0.17
50%,23.00,23.00,1.67,6.00,49.00,2.00,5.00,51.00,68.88,69.23,...,67.47,-0.20,74.79,74.72,0.20,0.00,1.80,1.87,0.25,0.27
75%,36.00,37.00,2.00,8.00,55.00,2.00,8.00,56.00,72.82,73.13,...,72.25,7.03,78.48,78.23,5.32,6.00,2.07,2.20,0.39,0.42
max,86.00,88.00,8.00,24.00,80.00,9.00,26.00,80.00,85.41,84.61,...,88.30,37.34,88.98,88.64,26.24,51.00,3.40,3.60,2.50,4.00


In [19]:
df_.to_csv('../data/preprocessed/preprocessed_1.csv', index=False)

In [20]:
df_.filter(like='points').describe().round(2)

,points_home,points_away,points_difference
count,2974.00,2974.00,2974.00
mean,25.66,25.89,-0.23
std,17.78,17.70,13.87
min,0.00,0.00,-54.00
25%,12.00,12.00,-7.00
50%,23.00,23.00,0.00
75%,36.00,37.00,6.00
max,86.00,88.00,51.00
